In [1]:
# CONFIGURATION
import os
BASE_DIR = os.path.expanduser(r"E:\DragonEye")
RAW_DATA_DIR = os.path.join(BASE_DIR, "raw_data")
OUTPUT_DIR = os.path.join(BASE_DIR, "dataset")
SEGMENTED_DIR = os.path.join(OUTPUT_DIR, "segmented")
FEATURE_CSV = os.path.join(OUTPUT_DIR, "features.csv")

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(SEGMENTED_DIR, exist_ok=True)

print("BASE_DIR:", BASE_DIR)
print("RAW_DATA_DIR:", RAW_DATA_DIR)
print("OUTPUT_DIR:", OUTPUT_DIR)
print("SEGMENTED_DIR:", SEGMENTED_DIR)
print("FEATURE_CSV:", FEATURE_CSV)

BASE_DIR: E:\DragonEye
RAW_DATA_DIR: E:\DragonEye\raw_data
OUTPUT_DIR: E:\DragonEye\dataset
SEGMENTED_DIR: E:\DragonEye\dataset\segmented
FEATURE_CSV: E:\DragonEye\dataset\features.csv


In [ ]:
# UTILS
import cv2
import os

def load_all_images(base_path):
    image_paths = []
    for root, _, files in os.walk(base_path):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_paths.append(os.path.join(root, file))
    image_paths.sort()
    return image_paths

def save_image(output_path, image):
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    cv2.imwrite(output_path, image)

In [ ]:
# PREPROCESSING
import cv2
import numpy as np

def preprocess_image(img):
    """Resize, denoise, dan konversi warna ke HSV"""
    img = cv2.resize(img, (256, 256))
    img = cv2.GaussianBlur(img, (3, 3), 0)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    return hsv

In [ ]:
# SEGMENTATION
import cv2
import numpy as np

def segment_image(hsv):
    """Segmentasi buah naga (mengembalikan segmented HSV dan mask biner)"""
    lower_red1 = np.array([0, 40, 40], dtype=np.uint8)
    upper_red1 = np.array([15, 255, 255], dtype=np.uint8)
    lower_red2 = np.array([160, 40, 40], dtype=np.uint8)
    upper_red2 = np.array([180, 255, 255], dtype=np.uint8)
    lower_green = np.array([35, 40, 40], dtype=np.uint8)
    upper_green = np.array([90, 255, 255], dtype=np.uint8)
    lower_yellow = np.array([20, 40, 40], dtype=np.uint8)
    upper_yellow = np.array([45, 255, 255], dtype=np.uint8)

    mask_red = cv2.bitwise_or(cv2.inRange(hsv, lower_red1, upper_red1),
                              cv2.inRange(hsv, lower_red2, upper_red2))
    mask_green = cv2.inRange(hsv, lower_green, upper_green)
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)
    mask = cv2.bitwise_or(mask_red, cv2.bitwise_or(mask_green, mask_yellow))

    bg_light = cv2.inRange(hsv, np.array([0, 0, 160], dtype=np.uint8), np.array([180, 60, 255], dtype=np.uint8))
    bg_dark = cv2.inRange(hsv, np.array([0, 0, 0], dtype=np.uint8), np.array([180, 100, 50], dtype=np.uint8))
    bg_mask = cv2.bitwise_or(bg_light, bg_dark)
    mask = cv2.bitwise_and(mask, cv2.bitwise_not(bg_mask))

    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)

    edge_refine = cv2.inRange(hsv, np.array([0, 0, 130], dtype=np.uint8), np.array([180, 70, 255], dtype=np.uint8))
    edge_refine = cv2.GaussianBlur(edge_refine, (5, 5), 0)
    edge_refine = cv2.dilate(edge_refine, kernel, iterations=1)
    mask = cv2.bitwise_and(mask, cv2.bitwise_not(edge_refine))

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        filled_mask = np.zeros_like(mask)
        cv2.drawContours(filled_mask, [max(contours, key=cv2.contourArea)], -1, 255, -1)
        mask = filled_mask

    mask_blur = cv2.GaussianBlur(mask, (3, 3), 0)
    _, mask_final = cv2.threshold(mask_blur, 100, 255, cv2.THRESH_BINARY)

    segmented = cv2.bitwise_and(hsv, hsv, mask=mask_final)
    return segmented, mask_final

In [ ]:
# FEATURE EXTRACTION
import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops

def extract_features(segmented_img, mask):
    """Ekstraksi fitur buah naga:
    Return: area (pixel), width, height, weight_est, texture_score, hue_mean
    """
    if mask is None or np.count_nonzero(mask) == 0:
        return 0.0, 0, 0, 0.0, 0.0, 0.0

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return 0.0, 0, 0, 0.0, 0.0, 0.0

    c = max(contours, key=cv2.contourArea)
    area = float(cv2.contourArea(c))
    x, y, w_box, h_box = cv2.boundingRect(c)
    w_box, h_box = int(w_box), int(h_box)

    k = 0.004
    weight_est = k * area

    hsv = segmented_img.copy()
    if len(hsv.shape) == 3:
        h_ch, s_ch, v_ch = cv2.split(hsv)
    else:
        h_ch = s_ch = v_ch = hsv

    x0, y0 = max(0, x), max(0, y)
    x1, y1 = min(hsv.shape[1], x + w_box), min(hsv.shape[0], y + h_box)
    s_crop = s_ch[y0:y1, x0:x1]
    h_crop = h_ch[y0:y1, x0:x1]
    mask_crop = mask[y0:y1, x0:x1]

    if mask_crop is None or mask_crop.size == 0 or np.count_nonzero(mask_crop) == 0:
        hue_mean = float(np.mean(h_ch[mask > 0]) / 180.0) if np.count_nonzero(mask) > 0 else 0.0
        return area, w_box, h_box, weight_est, 0.0, hue_mean

    region_s = s_crop[mask_crop > 0]
    region_h = h_crop[mask_crop > 0]

    if region_s.size == 0:
        hue_mean = float(np.mean(region_h) / 180.0) if region_h.size > 0 else 0.0
        return area, w_box, h_box, weight_est, 0.0, hue_mean

    levels = 64
    s_norm = cv2.normalize(s_crop, None, 0, levels - 1, cv2.NORM_MINMAX).astype(np.uint8)
    s_masked = np.where(mask_crop > 0, s_norm, 0).astype(np.uint8)

    if np.count_nonzero(mask_crop) < 10:
        contrast = homogeneity = energy = 0.0
    else:
        try:
            glcm = graycomatrix(
                s_masked,
                distances=[1, 2],
                angles=[0, np.pi/4, np.pi/2],
                levels=levels,
                symmetric=True,
                normed=True
            )
            contrast = float(np.mean(graycoprops(glcm, 'contrast')))
            homogeneity = float(np.mean(graycoprops(glcm, 'homogeneity')))
            energy = float(np.mean(graycoprops(glcm, 'energy')))
        except Exception:
            contrast = homogeneity = energy = 0.0

    texture_score = (homogeneity + energy) / 2.0 * (1.0 - contrast / (contrast + 1.0))
    hue_mean = float(np.mean(region_h) / 180.0) if region_h.size > 0 else 0.0

    return area, w_box, h_box, weight_est, float(texture_score), hue_mean

In [ ]:
# MAIN PIPELINE (Tidak dijalankan otomatis)
import csv
from IPython.display import display
from pathlib import Path

def run_pipeline(dry_run=True, max_items=None):
    image_paths = load_all_images(RAW_DATA_DIR)
    print(f"[INFO] Total gambar ditemukan: {len(image_paths)}")
    rows = []
    for idx, path in enumerate(image_paths):
        if max_items and idx >= max_items:
            break
        img_name = Path(path).name
        print(f"[PROCESS] Memproses: {img_name}")
        img = cv2.imread(path)
        if img is None:
            print(f" ⚠️ Gagal membaca {img_name}")
            continue
        hsv = preprocess_image(img)
        segmented, mask = segment_image(hsv)
        area, width, height, weight, texture_score, hue_mean = extract_features(segmented, mask)

        # Simpan segmented preview jika tidak dry_run
        if not dry_run:
            try:
                out_bgr = cv2.cvtColor(segmented, cv2.COLOR_HSV2BGR)
            except:
                out_bgr = cv2.cvtColor(preprocess_image(img), cv2.COLOR_HSV2BGR)
            out_path = os.path.join(SEGMENTED_DIR, img_name)
            save_image(out_path, out_bgr)

        rows.append([img_name, area, width, height, weight, texture_score, hue_mean])

    # Tulis CSV (jika bukan dry_run)
    if not dry_run:
        with open(FEATURE_CSV, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["filename", "area", "width", "height", "weight_est", "texture_score", "hue_mean"])
            writer.writerows(rows)
        print("[OK] Hasil disimpan ke:", FEATURE_CSV)
    else:
        print("[DRY RUN] Total baris yang diproses:", len(rows))
        display(rows[:10])
    return rows

# Untuk mencoba jalankan sedikit (dry_run=True hanya mengekstrak fitur tanpa menyimpan file)
# rows = run_pipeline(dry_run=True, max_items=5)

In [ ]:
# FUZZY GRADING (Area mentah dinormalisasi di sini)
import pandas as pd
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

def normalize(series):
    s_min, s_max = series.min(), series.max()
    if s_max == s_min:
        return np.zeros_like(series)
    return np.clip((series - s_min) / (s_max - s_min), 0, 1)

def normalize_texture(series):
    eps = 1e-6
    shifted = series - series.min() + eps
    log_norm = np.log1p(shifted)
    denom = log_norm.max() - log_norm.min()
    if denom == 0:
        return np.zeros_like(series)
    log_norm = (log_norm - log_norm.min()) / denom
    return np.clip(log_norm, 0, 1)

def fuzzy_grade_features(df):
    required = ['area', 'weight_est', 'texture_score']
    for col in required:
        if col not in df.columns:
            raise ValueError(f"Kolom {col} tidak ada di dataframe")

    df['area_norm'] = normalize(df['area'])
    df['weight_norm'] = normalize(df['weight_est'])
    df['texture_norm'] = normalize_texture(df['texture_score'])

    ukuran = ctrl.Antecedent(np.arange(0, 1.01, 0.01), 'ukuran')
    berat = ctrl.Antecedent(np.arange(0, 1.01, 0.01), 'berat')
    tekstur = ctrl.Antecedent(np.arange(0, 1.01, 0.01), 'tekstur')
    grade_out = ctrl.Consequent(np.arange(0, 101, 1), 'grade_out')

    ukuran['kecil'] = fuzz.trimf(ukuran.universe, [0.0, 0.0, 0.35])
    ukuran['sedang'] = fuzz.trimf(ukuran.universe, [0.3, 0.55, 0.7])
    ukuran['besar']  = fuzz.trimf(ukuran.universe, [0.6, 0.9, 1.0])

    berat['rendah']  = fuzz.trimf(berat.universe, [0.0, 0.0, 0.35])
    berat['sedang']  = fuzz.trimf(berat.universe, [0.3, 0.55, 0.7])
    berat['tinggi']  = fuzz.trimf(berat.universe, [0.6, 0.9, 1.0])

    tekstur['kasar'] = fuzz.trimf(tekstur.universe, [0.0, 0.0, 0.25])
    tekstur['normal']= fuzz.trimf(tekstur.universe, [0.2, 0.45, 0.7])
    tekstur['halus'] = fuzz.trimf(tekstur.universe, [0.4, 0.7, 1.0])

    grade_out['C'] = fuzz.trimf(grade_out.universe, [0, 0, 40])
    grade_out['B'] = fuzz.trimf(grade_out.universe, [35, 55, 75])
    grade_out['A'] = fuzz.trimf(grade_out.universe, [60, 100, 100])

    rules = [
        ctrl.Rule(ukuran['besar'] & berat['tinggi'], grade_out['A']),
        ctrl.Rule(ukuran['sedang'] & berat['tinggi'], grade_out['A']),
        ctrl.Rule(ukuran['besar'] & tekstur['normal'], grade_out['A']),
        ctrl.Rule(berat['tinggi'] & tekstur['halus'], grade_out['A']),
        ctrl.Rule(ukuran['besar'] & tekstur['halus'], grade_out['A']),
        ctrl.Rule(ukuran['sedang'] & berat['sedang'] & tekstur['halus'], grade_out['A']),
        ctrl.Rule(ukuran['sedang'] & berat['sedang'], grade_out['B']),
        ctrl.Rule(ukuran['besar'] & tekstur['kasar'], grade_out['B']),
        ctrl.Rule(ukuran['sedang'] & tekstur['normal'], grade_out['B']),
        ctrl.Rule(berat['rendah'] & tekstur['halus'], grade_out['B']),
        ctrl.Rule(ukuran['besar'] & berat['rendah'], grade_out['B']),
        ctrl.Rule(ukuran['kecil'] & berat['tinggi'], grade_out['B']),
        ctrl.Rule(ukuran['kecil'] | berat['rendah'], grade_out['C']),
        ctrl.Rule(tekstur['kasar'], grade_out['C']),
    ]

    grading_ctrl = ctrl.ControlSystem(rules)

    grades = []
    for _, row in df.iterrows():
        sim = ctrl.ControlSystemSimulation(grading_ctrl)
        sim.input['ukuran'] = float(row['area_norm'])
        sim.input['berat'] = float(row['weight_norm'])
        sim.input['tekstur'] = float(row['texture_norm'])
        try:
            sim.compute()
            score = sim.output['grade_out']
        except Exception:
            score = 0
        label = 'A' if score >= 60 else ('B' if score >= 40 else 'C')
        grades.append((score, label))

    df['grade_score'] = [g[0] for g in grades]
    df['grade_label'] = [g[1] for g in grades]
    return df

In [ ]:
# SAVE NOTE
print("Notebook siap. Simpan / jalankan sesuai kebutuhan lokal Anda.")